## Load Data Module Test

- k-means
- locations
- sequences


In [45]:
# Test
import sys
import pandas as pd
    
sys.path.append("../new_codebase/src/utils/")
from connect_db import db_connection

#sys.path.append("../new_codebase/src/utils/load_data/")
#from load_dataframes import *

In [2]:
from datetime import datetime
def calc_season(season):
    
    """
    Input: a season (pre-summer, summer, post-summer, winter, all)
    Returns: start and end date as strings
        min_date: minimum date, in the yyyy-mm-dd hh:mm:ss format
        max_date: maximum date, in the same format
    """
    if season == 'pre-summer':
        return (str(datetime(2017, 5, 1, hour=0, minute=0, second=0)),
                str(datetime(2017, 5, 31, hour=23, minute=59, second=59)))
    elif season == "summer":
        return (str(datetime(2017, 6, 1, hour=0, minute=0, second=0)),
                str(datetime(2017, 8, 31, hour=23, minute=59, second=59)))
    elif season == "post-summer":
        return (str(datetime(2017, 9, 1, hour=0, minute=0, second=0)),
                str(datetime(2017, 10, 31, hour=23, minute=59, second=59)))
    elif season == "winter":
        return (str(datetime(2017, 11, 1, hour=0, minute=0, second=0)),
                str(datetime(2018, 2, 28, hour=23, minute=59, second=59)))
    elif season == "all":
        return (str(datetime(2017, 5, 1, hour=0, minute=0, second=0)),
                str(datetime(2018, 2, 28, hour=23, minute=59, second=59)))
    else:
        print ("Do you think {} matters? Are you not working in tourism???? You teapot!!!".format(season))
        print ("Give me a real season, choose one of these: pre-summer, summer, post-summer, winter")

In [3]:
def create_connection(username):
    """
    Create connection with tha database
    Data_creds_redshift.json.nogit should be places under /mnt/data/{username}/utils/
    """
    
    cred_location = "/mnt/data/{}/utils/data_creds_redshift.json.nogit".format(username)
    db = db_connection.DBConnection(cred_location)
    return db

In [4]:
def get_location_sequences(username, country, min_date, max_date):
    """
    Run the query for location sequencess of a given country in a given time window
    
    Params:
        country: name of the country (for more details see mcc table)
        min_date: minimum date, in the yyyy-mm-dd hh:mm:ss format
        max_date: maximum date, in the same format
        
    Returns:
        df_location_sequence: feature table of a given country in a given season
    """
    import pandas as pd
    
    db=create_connection(username)
    country=country.title()

    query=f'with get_mcc_country as  \
        (select * from tuscany.customer_feature cus  \
            join tuscany.mcc mc  \
        on mc.mcc=cus.mcc \
        where mc.country={country!r}  \
            and (cus.st_time between {min_date!r} and {max_date!r})) \
             \
    select com_locs_trunc from tuscany.customer_arrays ars \
        inner join get_mcc_country feat \
    on feat.customer_id = ars.customer_id \
    and com_locs_trunc is not null'

    df = db.sql_query_to_data_frame(query, cust_id=True)
    db.engine.dispose()
    
    return df

In [5]:
def get_geo2vec_data(username, season, country):
    """
    Params:
        username: to connect redshift,  
                  data_creds_redshift.json.nogit should be places under /mnt/data/{username}/utils/
        country: str, name of the country (for more details see mcc table)
        season: str, pre-summer, summer, post-summer, winter
    """
    min_date, max_date = calc_season(season)
    df = get_location_sequences(username, country, min_date, max_date)
    return df

In [6]:
def get_geo2vec_data_all_country(username, season):
    """
    Params:
        username: to connect redshift,  
                  data_creds_redshift.json.nogit should be places under /mnt/data/{username}/utils/
        country: str, name of the country (for more details see mcc table)
        season: str, pre-summer, summer, post-summer, winter
    """
    min_date, max_date = calc_season(season)
    db=create_connection(username)

    query=f'with get_mcc_country as  \
        (select * from tuscany.customer_feature cus  \
         where (cus.st_time between {min_date!r} and {max_date!r})) \
             \
        select com_locs_trunc from tuscany.customer_arrays ars \
        inner join get_mcc_country feat \
        on feat.customer_id = ars.customer_id \
        and com_locs_trunc is not null'

    df = db.sql_query_to_data_frame(query, cust_id=True)
    db.engine.dispose()
        
    return df

In [7]:
df3=get_geo2vec_data_all_country('ovasarhelyi','winter')

In [8]:
df3.tail()

,com_locs_trunc
1041541,"50009, 50009, 50033, 50009, 50033, 50009, 5003..."
1041542,"49007, 53001, 49001, 49001, 49007, 49007, 4900..."
1041543,"52015, 52009, 52015"
1041544,"50036, 46017"
1041545,"96041, 50039, 53011, 49007, 13251"


In [9]:
df2=get_geo2vec_data('ovasarhelyi','winter', 'hungary')

In [10]:
df2.tail()

,com_locs_trunc
7170,"48043, 48017, 37006, 48043, 37051, 100006, 370..."
7171,"48024, 48017, 48028"
7172,"48053, 48043, 48022, 48038, 48017, 48022, 4803..."
7173,"48038, 52022, 48017, 48014"
7174,"46033, 50026"


In [11]:
calc_season("winter")

('2017-11-01 00:00:00', '2018-02-28 23:59:59')

In [12]:
def get_customer_arrays(username, country, min_date, max_date, min_trip, max_trip):
    """
    Run the query for customer arrays of a given country in a given time window
    
    Params:
        country: name of the country (for more details see mcc table), 
            et country = "all:" means all countries
        min_date: minimum date, in the yyyy-mm-dd hh:mm:ss format
        max_date: maximum date, in the same format
        min_trip: int, filter for customers with the given minimum length of trip duration in Italy
        max_trip: int, filter for customers with the given maximum length of trip duration in Italy
        
    Returns:
        df_location_sequence: feature table of a given country in a given season
    """
    import pandas as pd

    db=create_connection(username)
    country=country.title()

    query=f'select  \
        ars.customer_nr,  \
        ars.com_locs_new as locations,  \
        ars.times_new as times,  \
        ars.st_time,  \
        ars.en_time,  \
        ars.mcc  \
        from tuscany.customer_arrays ars  \
        where  ars.trip_duration <= {max_trip!r}  \
        and ars.trip_duration > {min_trip!r}  \
        and times_new is not null  \
        and (st_time between {min_date!r} and {max_date!r})  \
        and mcc in (select mcc from tuscany.mcc mc  \
        where mc.country={country!r})'
    
    df = db.sql_query_to_data_frame(query, cust_id=False)
    db.engine.dispose()
    return df

In [13]:
def get_sequence_data(username, season, country, min_trip, max_trip):
    """
    Params:
        username: to connect redshift,  
                  data_creds_redshift.json.nogit should be places under /mnt/data/{username}/utils/
        country: str, name of the country (for more details see mcc table)
        season: str, pre-summer, summer, post-summer, winter
    """
    min_date, max_date = calc_season(season)
    df = get_customer_arrays(username, country, min_date, max_date, min_trip, max_trip)
    return df

In [14]:
df3=get_sequence_data('ovasarhelyi', 'winter', 'Germany', 1, 2)

In [15]:
df3.head()

,customer_nr,locations,times,st_time,en_time,mcc
0,8651429,"47014, 47020, 47014","1, 1759",2017-11-03 14:57:00,2017-11-04 20:20:00,262
1,3571311,"56050, 58097, 58032, 56035, 58032, 58097, 5803...","33, 24, 1, 82, 1, 58, 12, 1, 1, 5, 11, 118, 2,...",2017-11-13 06:49:00,2017-11-14 21:25:00,262
2,3666356,"13075, 13154, 13245, 13036, 13133, 13227, 1511...","2, 2, 3, 3, 1, 5, 3, 8, 1, 2, 4, 49, 30, 13, 5...",2017-12-04 14:10:00,2017-12-06 14:17:00,262
3,3670753,"11030, 11019, 11030, 11019, 11030, 11019, 1103...","38, 22, 12, 7, 1, 2, 1, 1, 196, 11, 10, 9, 2, ...",2017-12-03 00:05:00,2017-12-04 13:38:00,262
4,8756129,"46035, 50026, 49009, 50026, 49009, 50026, 4900...","315, 3, 1, 1, 5, 859, 31, 122, 57, 359, 7, 3, ...",2018-02-27 01:18:00,2018-02-28 20:59:00,262


In [16]:
min_t=1
max_t=10
min_t1=8
max_t1=10

In [17]:
df4=get_sequence_data('bdelpapa','winter', 'Hungary',1,20)

In [18]:
len(df4)

5210

In [19]:
df5=get_sequence_data('ovasarhelyi','winter', 'Germany', min_trip=min_t1, max_trip=max_t1)

In [20]:
len(df5)

1519

In [21]:
def get_customer_features(username, country, min_date, max_date):
    """
    Run the query for customer features of a given country in a given time window
    
    Params:
        country: name of the country (for more details see mcc table), 
            set country = "all" to get all all countries
        min_date: minimum date, in the yyyy-mm-dd hh:mm:ss format
        max_date: maximum date, in the same format
        
    Returns:
        df_location_sequence: feature table of a given country in a given season
    """
    db=create_connection(username)
    country=country.title()
    
    query=f'select * from tuscany.customer_feature cus  \
                join tuscany.mcc mc  \
            on mc.mcc=cus.mcc  \
            where mc.country={country!r}  \
                and (cus.st_time between {min_date!r} and {max_date!r})  \
                and cus.customer_id not in (select customer_id from tuscany.excluded_customers)' 
    df = db.sql_query_to_data_frame(query, cust_id=False)
    db.engine.dispose()
    return df

In [38]:
def digitalize_features(df):
    """
    digitalize numeric columns for kmeans
    """
    for i in df.columns:
        if i.lower() != 'country' and i!='customer_id':
            df[i]=pd.to_numeric(df[i])
    return df

In [39]:
def get_k_means_data_for_all_countries(username, season):
    """ 
    Uset his function to get k-means data for all countries
    Params:
        username: to connect redshift,  
                  data_creds_redshift.json.nogit should be places under /mnt/data/{username}/utils/
        season: str, pre-summer, summer, post-summer, winter
    """
    min_date, max_date = calc_season(season)
    db=create_connection(username)
    query=f'select * from tuscany.customer_feature cus  \
                join tuscany.mcc mc  \
            on mc.mcc=cus.mcc  \
                and (cus.st_time between {min_date!r} and {max_date!r})  \
                and cus.customer_id not in (select customer_id from tuscany.excluded_customers)'
    df = db.sql_query_to_data_frame(query, cust_id=True)
    db.engine.dispose()
    df2 = digitalize_features(df)
    return df2

In [40]:
def get_k_means_data(username, season, country):
    """
    Params:
        username: to connect redshift,  
                  data_creds_redshift.json.nogit should be places under /mnt/data/{username}/utils/
        country: str, name of the country (for more details see mcc table)
        season: str, pre-summer, summer, post-summer, winter
    """
    min_date, max_date = calc_season(season)
    df = get_customer_features(username, country, min_date, max_date)
    return df

In [41]:
df=get_k_means_data('ovasarhelyi', 'winter', 'hungary')

In [42]:
len(df)

7175

In [46]:
df2=get_k_means_data_for_all_countries('ovasarhelyi', 'pre-summer')
len(df2)

725525